In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
india_stations_pm25 = pd.read_pickle("./Data/India/india_stations_pm25.pkl")

In [3]:
def impute(station_df, column="PM2.5", window_length=24):
    try:
        for idx, value in enumerate(station_df[column].values):
            if np.isnan(value):
                station_df[column][idx] = station_df[column][idx - window_length]
    except:
        pass
    return station_df

In [4]:
station_data = []

for station_df in india_stations_pm25:
    length = station_df.shape[0]
    station_df = station_df["2018-01-01":]
    if (station_df.isna().sum()['PM2.5']/station_df.shape[0] > 0.3):
        continue
    station_df = impute(station_df, column="PM2.5", window_length=24)
    
    if length > 500:    
        station_data.append((length, station_df))
    
station_data.sort(key=lambda x:x[0], reverse=True)

In [5]:
# explored_states = []
max_length_dfs = []

for length, station_df in station_data:
    station_code = station_df['StationId'].values[0]
    state_code = station_code[:2]
#     if state_code not in explored_states:
#     explored_states.append(state_code)
    max_length_dfs.append((station_code, station_df))

In [6]:
india_stations_df = pd.read_csv("./Data/India/india_stations-corrected.csv", index_col=0)
india_stations_df.head()

,StationName,StationId,City,Latitude,Longitude
0,"Secretariat, Amaravati - APPCB",AP001,Amaravati,16.504561,80.523517
1,"GVM Corporation, Visakhapatnam - APPCB",AP005,Visakhapatnam,17.748887,83.247789
2,"Railway Colony, Guwahati - APCB",AS001,Guwahati,26.179587,91.784315
3,"DRM Office Danapur, Patna - BSPCB",BR005,Patna,25.585624,85.044299
4,"Govt. High School Shikarpur, Patna - BSPCB",BR006,Patna,25.587980,85.094986


In [7]:
dataset = []

for station_code, df in max_length_dfs:
    details = india_stations_df[india_stations_df['StationId'] == station_code]
    station_name, station_id, city, latitude, longitude = details.values[0]
    
    if (city == "Delhi"):
        
        if station_id == "DL013":
            continue

        item = {
            "station_name": station_name,
            "station_id": station_id, 
            "city": city,
            "latitude": latitude,
            "longitude": longitude,
            "df": df[['PM2.5']]
        }

        dataset.append(item)

In [9]:
dataset = dataset[:8]

In [10]:
import pickle

pickle.dump(dataset, open("./Data/delhi_dataset.pkl", "wb"), protocol=4)